In [2]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Utils.utils import create_URM, create_ICM, combine_matrices
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from skopt.space import Real
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

In [ ]:
##CREATE URM,ICM, ICM STACKED COMPLETELY TO TRAIN THE SINGLE RECOMMENDERS, AND STACKED PARTIALLY TO TUNE THE HYBRID
URM = create_URM()
ICM = create_ICM()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
combined_matrices = combine_matrices(ICM=ICM, URM=URM_train)
ICM_combined = combine_matrices(ICM=ICM,URM=URM)

In [ ]:
### UserCF Recommender
UserCF_recommender = UserKNNCFRecommender(URM_train = ICM_combined)
UserCF_recommender.fit(topK= 450, shrink= 5.0, similarity= "tanimoto")

In [ ]:
## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=ICM_combined)
IALS_recommender.fit(epochs=50, num_factors=40, alpha=0.7917156202136513, reg=1.0)

In [ ]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=ICM_combined)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

In [4]:
##HYPERPARAMETER TUNING OF HYBRID
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

hyperparameters_range_dictionary = {
    "alpha": Real(0.0, 1.0),
    "beta": Real(0.0, 1.0),
    "gamma": Real(0.0, 1.0)
}
recommenders = [UserCF_recommender, IALS_recommender, SLIME_recommender]

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[combined_matrices, recommenders],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=MergeThreeModelsByScores,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 100
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=MergeThreeModelsByScores.RECOMMENDER_NAME,
                    save_model="best"
                    )
## Best config is 56: {'alpha': 0.9236610693606594, 'beta': 0.31737234692557365, 'gamma': 0.01980260938401746}



Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.20880328503235251, 'beta': 0.7808193342016669, 'gamma': 0.37693175782518173}
EvaluatorHoldout: Processed 13632 (100.0%) in 20.56 sec. Users per second: 663
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.20880328503235251, 'beta': 0.7808193342016669, 'gamma': 0.37693175782518173} - results: PRECISION: 0.2607468, PRECISION_RECALL_MIN_DEN: 0.2666073, RECALL: 0.0792112, MAP: 0.1347140, MAP_MIN_DEN: 0.1378443, MRR: 0.5146098, NDCG: 0.2762837, F1: 0.1215095, HIT_RATE: 0.9408011, ARHR_ALL_HITS: 0.8297298, NOVELTY: 0.0054336, AVERAGE_POPULARITY: 0.6019632, DIVERSITY_MEAN_INTER_LIST: 0.8474482, DIVERSITY_HERFINDAHL: 0.9847386, COVERAGE_ITEM: 0.0575336, COVERAGE_ITEM_CORRECT: 0.0369345, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9395604, DIVERSITY_GINI: 0.0064957, SHANNON_ENTROPY: 7.0502823, RATIO_DIVERSITY_HERFINDAHL: 0.9851085, RATIO_DIVERSITY_GINI: 0.025057

100%|████████████████████████████████████▉| 18056/18059 [17:00<00:00, 17.69it/s]


EvaluatorHoldout: Processed 13632 (100.0%) in 20.32 sec. Users per second: 671
SearchBayesianSkopt: Config 3 is suboptimal. Config: {'alpha': 0.25961695625355813, 'beta': 0.6330502541503795, 'gamma': 0.3492875125297596} - results: PRECISION: 0.2615317, PRECISION_RECALL_MIN_DEN: 0.2676658, RECALL: 0.0801293, MAP: 0.1352729, MAP_MIN_DEN: 0.1385956, MRR: 0.5160057, NDCG: 0.2774159, F1: 0.1226734, HIT_RATE: 0.9425616, ARHR_ALL_HITS: 0.8328570, NOVELTY: 0.0054318, AVERAGE_POPULARITY: 0.6036519, DIVERSITY_MEAN_INTER_LIST: 0.8451616, DIVERSITY_HERFINDAHL: 0.9845100, COVERAGE_ITEM: 0.0585304, COVERAGE_ITEM_CORRECT: 0.0369345, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9413187, DIVERSITY_GINI: 0.0063621, SHANNON_ENTROPY: 7.0201332, RATIO_DIVERSITY_HERFINDAHL: 0.9848798, RATIO_DIVERSITY_GINI: 0.0245425, RATIO_SHANNON_ENTROPY: 0.5641440, RATIO_AVERAGE_POPULARITY: 3.0424691, RATIO_NOVELTY: 0.0284145, 

Iteration No: 4 ended. Evaluation done at random point.
Time taken: 20.4460
Function val

EvaluatorHoldout: Processed 13639 (100.0%) in 20.25 sec. Users per second: 673
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'alpha': 0.7350895808404728, 'beta': 0.21551727741975796, 'gamma': 0.11521355965931326} - results:
CUTOFF: 10 - PRECISION: 0.3102060, PRECISION_RECALL_MIN_DEN: 0.3161484, RECALL: 0.0831595, MAP: 0.1720000, MAP_MIN_DEN: 0.1755195, MRR: 0.5764373, NDCG: 0.3283583, F1: 0.1311583, HIT_RATE: 0.9667131, ARHR_ALL_HITS: 0.9882648, NOVELTY: 0.0054396, AVERAGE_POPULARITY: 0.6038672, DIVERSITY_MEAN_INTER_LIST: 0.8428863, DIVERSITY_HERFINDAHL: 0.9842825, COVERAGE_ITEM: 0.0744781, COVERAGE_ITEM_CORRECT: 0.0438009, COVERAGE_USER: 0.9991941, COVERAGE_USER_CORRECT: 0.9659341, DIVERSITY_GINI: 0.0064203, SHANNON_ENTROPY: 6.9984454, RATIO_DIVERSITY_HERFINDAHL: 0.9846522, RATIO_DIVERSITY_GINI: 0.0247668, RATIO_SHANNON_ENTROPY: 0.5624011, RATIO_AVERAGE_POPULARITY: 3.0435538, RATIO_NOVELTY: 0.0284553, 


SearchBayesianSkopt: Saving model in logs/MergeThreeModelsB

EvaluatorHoldout: Processed 13632 (100.0%) in 20.05 sec. Users per second: 680
SearchBayesianSkopt: Config 16 is suboptimal. Config: {'alpha': 0.0363385382908682, 'beta': 0.9094661862971383, 'gamma': 0.8529769528308663} - results: PRECISION: 0.2543941, PRECISION_RECALL_MIN_DEN: 0.2591612, RECALL: 0.0743101, MAP: 0.1282095, MAP_MIN_DEN: 0.1305030, MRR: 0.4914994, NDCG: 0.2656223, F1: 0.1150216, HIT_RATE: 0.9343457, ARHR_ALL_HITS: 0.7915656, NOVELTY: 0.0054882, AVERAGE_POPULARITY: 0.5697937, DIVERSITY_MEAN_INTER_LIST: 0.8816519, DIVERSITY_HERFINDAHL: 0.9881587, COVERAGE_ITEM: 0.0700482, COVERAGE_ITEM_CORRECT: 0.0442992, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9331136, DIVERSITY_GINI: 0.0089496, SHANNON_ENTROPY: 7.5056095, RATIO_DIVERSITY_HERFINDAHL: 0.9885299, RATIO_DIVERSITY_GINI: 0.0345237, RATIO_SHANNON_ENTROPY: 0.6031573, RATIO_AVERAGE_POPULARITY: 2.8718201, RATIO_NOVELTY: 0.0287099, 

Iteration No: 17 ended. Evaluation done at random point.
Time taken: 20.1671
Function va

...

Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 21.1951
Function value obtained: -0.1364
Current minimum: -0.1365
Iteration No: 72 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.7649591973207889, 'beta': 0.39790054750600345, 'gamma': 0.08042172597786947}
EvaluatorHoldout: Processed 13632 (100.0%) in 20.27 sec. Users per second: 672
SearchBayesianSkopt: Config 71 is suboptimal. Config: {'alpha': 0.7649591973207889, 'beta': 0.39790054750600345, 'gamma': 0.08042172597786947} - results: PRECISION: 0.2627861, PRECISION_RECALL_MIN_DEN: 0.2699020, RECALL: 0.0827433, MAP: 0.1363847, MAP_MIN_DEN: 0.1403043, MRR: 0.5195841, NDCG: 0.2798147, F1: 0.1258579, HIT_RATE: 0.9449824, ARHR_ALL_HITS: 0.8394279, NOVELTY: 0.0054351, AVERAGE_POPULARITY: 0.6048360, DIVERSITY_MEAN_INTER_LIST: 0.8425013, DIVERSITY_HERFINDAHL: 0.9842440, COVERAGE_ITEM: 0.0666704, COVERAGE_ITEM_CORRECT: 0.0391495, COVERAGE_USER: 0.9986813, COVERAGE_U

Iteration No: 78 ended. Search finished for the next optimal point.
Time taken: 20.8266
Function value obtained: -0.1205
Current minimum: -0.1365
Iteration No: 79 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.9569259304781659, 'beta': 0.4893011545871948, 'gamma': 0.9578035244899282}
EvaluatorHoldout: Processed 13632 (100.0%) in 20.31 sec. Users per second: 671
SearchBayesianSkopt: Config 78 is suboptimal. Config: {'alpha': 0.9569259304781659, 'beta': 0.4893011545871948, 'gamma': 0.9578035244899282} - results: PRECISION: 0.2625807, PRECISION_RECALL_MIN_DEN: 0.2691808, RECALL: 0.0816324, MAP: 0.1358689, MAP_MIN_DEN: 0.1394305, MRR: 0.5161374, NDCG: 0.2786185, F1: 0.1245455, HIT_RATE: 0.9436620, ARHR_ALL_HITS: 0.8352135, NOVELTY: 0.0054368, AVERAGE_POPULARITY: 0.6038883, DIVERSITY_MEAN_INTER_LIST: 0.8431632, DIVERSITY_HERFINDAHL: 0.9843101, COVERAGE_ITEM: 0.0697159, COVERAGE_ITEM_CORRECT: 0.0392602, COVERAGE_USER: 0.9986813, COVERAGE_USER_

Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 23.3509
Function value obtained: -0.1363
Current minimum: -0.1365
Iteration No: 86 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.9990019720148339, 'beta': 0.7092680230858633, 'gamma': 0.9026582347080536}
EvaluatorHoldout: Processed 13632 (100.0%) in 20.68 sec. Users per second: 659
SearchBayesianSkopt: Config 85 is suboptimal. Config: {'alpha': 0.9990019720148339, 'beta': 0.7092680230858633, 'gamma': 0.9026582347080536} - results: PRECISION: 0.2626467, PRECISION_RECALL_MIN_DEN: 0.2692566, RECALL: 0.0816248, MAP: 0.1359766, MAP_MIN_DEN: 0.1395574, MRR: 0.5166746, NDCG: 0.2787696, F1: 0.1245441, HIT_RATE: 0.9438087, ARHR_ALL_HITS: 0.8358181, NOVELTY: 0.0054347, AVERAGE_POPULARITY: 0.6043781, DIVERSITY_MEAN_INTER_LIST: 0.8429809, DIVERSITY_HERFINDAHL: 0.9842919, COVERAGE_ITEM: 0.0664489, COVERAGE_ITEM_CORRECT: 0.0385957, COVERAGE_USER: 0.9986813, COVERAGE_USER_

Iteration No: 92 ended. Search finished for the next optimal point.
Time taken: 21.6728
Function value obtained: -0.1361
Current minimum: -0.1365
Iteration No: 93 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.6761826866766106, 'beta': 0.12318566620389303, 'gamma': 0.0}
EvaluatorHoldout: Processed 13632 (100.0%) in 20.70 sec. Users per second: 659
SearchBayesianSkopt: Config 92 is suboptimal. Config: {'alpha': 0.6761826866766106, 'beta': 0.12318566620389303, 'gamma': 0.0} - results: PRECISION: 0.2625440, PRECISION_RECALL_MIN_DEN: 0.2696990, RECALL: 0.0828519, MAP: 0.1363237, MAP_MIN_DEN: 0.1403074, MRR: 0.5197039, NDCG: 0.2797731, F1: 0.1259556, HIT_RATE: 0.9455692, ARHR_ALL_HITS: 0.8395041, NOVELTY: 0.0054426, AVERAGE_POPULARITY: 0.6031933, DIVERSITY_MEAN_INTER_LIST: 0.8432256, DIVERSITY_HERFINDAHL: 0.9843164, COVERAGE_ITEM: 0.0802924, COVERAGE_ITEM_CORRECT: 0.0414198, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9443223, DIVERSIT

Iteration No: 99 ended. Search finished for the next optimal point.
Time taken: 21.6974
Function value obtained: -0.1347
Current minimum: -0.1365
Iteration No: 100 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.5919809554474691, 'beta': 0.6466679955380832, 'gamma': 0.9416792060124639}
EvaluatorHoldout: Processed 13632 (100.0%) in 20.47 sec. Users per second: 666
SearchBayesianSkopt: Config 99 is suboptimal. Config: {'alpha': 0.5919809554474691, 'beta': 0.6466679955380832, 'gamma': 0.9416792060124639} - results: PRECISION: 0.2621185, PRECISION_RECALL_MIN_DEN: 0.2683926, RECALL: 0.0807237, MAP: 0.1355423, MAP_MIN_DEN: 0.1389160, MRR: 0.5152224, NDCG: 0.2778465, F1: 0.1234339, HIT_RATE: 0.9431485, ARHR_ALL_HITS: 0.8332883, NOVELTY: 0.0054362, AVERAGE_POPULARITY: 0.6031267, DIVERSITY_MEAN_INTER_LIST: 0.8445170, DIVERSITY_HERFINDAHL: 0.9844455, COVERAGE_ITEM: 0.0668365, COVERAGE_ITEM_CORRECT: 0.0380973, COVERAGE_USER: 0.9986813, COVERAGE_USER

In [5]:
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
ItemCF_recommender = ItemKNNCFRecommender(URM_train = ICM_combined)
ItemCF_recommender.fit(topK=217, shrink=5.0, similarity="cosine")


Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 18059 (100.0%), 905.90 column/sec. Elapsed time 19.93 sec


In [6]:
##HYPERPARAMETER TUNING OF HYBRID
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

hyperparameters_range_dictionary = {
    "alpha": Real(0.0, 1.0),
    "beta": Real(0.0, 1.0),
    "gamma": Real(0.0, 1.0)
}
recommenders_item = [ItemCF_recommender, IALS_recommender, SLIME_recommender]

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[combined_matrices, recommenders_item],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=MergeThreeModelsByScores,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 100
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=MergeThreeModelsByScores.RECOMMENDER_NAME + "_item",
                    save_model="best"
                    )





Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.7449630271753703, 'beta': 0.4416431157313694, 'gamma': 0.1603665989680483}
EvaluatorHoldout: Processed 13632 (100.0%) in 21.16 sec. Users per second: 644
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.7449630271753703, 'beta': 0.4416431157313694, 'gamma': 0.1603665989680483} - results: PRECISION: 0.1797609, PRECISION_RECALL_MIN_DEN: 0.1810012, RECALL: 0.0458072, MAP: 0.0763851, MAP_MIN_DEN: 0.0767133, MRR: 0.3429434, NDCG: 0.1781433, F1: 0.0730098, HIT_RATE: 0.8238703, ARHR_ALL_HITS: 0.5123807, NOVELTY: 0.0053058, AVERAGE_POPULARITY: 0.6656725, DIVERSITY_MEAN_INTER_LIST: 0.7826008, DIVERSITY_HERFINDAHL: 0.9782543, COVERAGE_ITEM: 0.0449637, COVERAGE_ITEM_CORRECT: 0.0228695, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.8227839, DIVERSITY_GINI: 0.0034832, SHANNON_ENTROPY: 6.2019930, RATIO_DIVERSITY_HERFINDAHL: 0.9786218, RATIO_DIVERSITY_GINI: 0.0134369, R

EvaluatorHoldout: Processed 13639 (100.0%) in 21.63 sec. Users per second: 630
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'alpha': 0.3043266227555193, 'beta': 0.9336231641991904, 'gamma': 0.7123469490391381} - results:
CUTOFF: 10 - PRECISION: 0.2273920, PRECISION_RECALL_MIN_DEN: 0.2288262, RECALL: 0.0516787, MAP: 0.1052006, MAP_MIN_DEN: 0.1056785, MRR: 0.4071258, NDCG: 0.2263340, F1: 0.0842175, HIT_RATE: 0.8892148, ARHR_ALL_HITS: 0.6537848, NOVELTY: 0.0053188, AVERAGE_POPULARITY: 0.6583571, DIVERSITY_MEAN_INTER_LIST: 0.7963773, DIVERSITY_HERFINDAHL: 0.9796319, COVERAGE_ITEM: 0.0501135, COVERAGE_ITEM_CORRECT: 0.0272994, COVERAGE_USER: 0.9991941, COVERAGE_USER_CORRECT: 0.8884982, DIVERSITY_GINI: 0.0039309, SHANNON_ENTROPY: 6.3573033, RATIO_DIVERSITY_HERFINDAHL: 0.9799999, RATIO_DIVERSITY_GINI: 0.0151637, RATIO_SHANNON_ENTROPY: 0.5108784, RATIO_AVERAGE_POPULARITY: 3.3181888, RATIO_NOVELTY: 0.0278235, 


SearchBayesianSkopt: Saving model in logs/MergeThreeModelsByS

EvaluatorHoldout: Processed 13632 (100.0%) in 21.05 sec. Users per second: 648
SearchBayesianSkopt: Config 10 is suboptimal. Config: {'alpha': 0.6330333089370683, 'beta': 0.7154767632729031, 'gamma': 0.7251975460712407} - results: PRECISION: 0.1846464, PRECISION_RECALL_MIN_DEN: 0.1863704, RECALL: 0.0482138, MAP: 0.0792059, MAP_MIN_DEN: 0.0798153, MRR: 0.3525462, NDCG: 0.1837920, F1: 0.0764623, HIT_RATE: 0.8366344, ARHR_ALL_HITS: 0.5291196, NOVELTY: 0.0053092, AVERAGE_POPULARITY: 0.6638793, DIVERSITY_MEAN_INTER_LIST: 0.7863638, DIVERSITY_HERFINDAHL: 0.9786306, COVERAGE_ITEM: 0.0463481, COVERAGE_ITEM_CORRECT: 0.0235893, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.8355311, DIVERSITY_GINI: 0.0036124, SHANNON_ENTROPY: 6.2473313, RATIO_DIVERSITY_HERFINDAHL: 0.9789982, RATIO_DIVERSITY_GINI: 0.0139352, RATIO_SHANNON_ENTROPY: 0.5020410, RATIO_AVERAGE_POPULARITY: 3.3460215, RATIO_NOVELTY: 0.0277732, 

Iteration No: 11 ended. Evaluation done at random point.
Time taken: 21.1680
Function va

EvaluatorHoldout: Processed 13632 (100.0%) in 21.56 sec. Users per second: 632
SearchBayesianSkopt: Config 17 is suboptimal. Config: {'alpha': 0.8191681250112002, 'beta': 0.9481156060938171, 'gamma': 0.7783491449361608} - results: PRECISION: 0.1840229, PRECISION_RECALL_MIN_DEN: 0.1857199, RECALL: 0.0479485, MAP: 0.0789170, MAP_MIN_DEN: 0.0795176, MRR: 0.3518847, NDCG: 0.1832112, F1: 0.0760751, HIT_RATE: 0.8350205, ARHR_ALL_HITS: 0.5276251, NOVELTY: 0.0053087, AVERAGE_POPULARITY: 0.6640807, DIVERSITY_MEAN_INTER_LIST: 0.7859764, DIVERSITY_HERFINDAHL: 0.9785919, COVERAGE_ITEM: 0.0461820, COVERAGE_ITEM_CORRECT: 0.0232571, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.8339194, DIVERSITY_GINI: 0.0035937, SHANNON_ENTROPY: 6.2417902, RATIO_DIVERSITY_HERFINDAHL: 0.9789595, RATIO_DIVERSITY_GINI: 0.0138629, RATIO_SHANNON_ENTROPY: 0.5015957, RATIO_AVERAGE_POPULARITY: 3.3470367, RATIO_NOVELTY: 0.0277708, 

Iteration No: 18 ended. Evaluation done at random point.
Time taken: 21.6864
Function va

EvaluatorHoldout: Processed 13632 (100.0%) in 21.41 sec. Users per second: 637
SearchBayesianSkopt: Config 23 is suboptimal. Config: {'alpha': 0.30250234201094844, 'beta': 0.23244903760729646, 'gamma': 0.15047735555802347} - results: PRECISION: 0.1815141, PRECISION_RECALL_MIN_DEN: 0.1828260, RECALL: 0.0465561, MAP: 0.0773789, MAP_MIN_DEN: 0.0777973, MRR: 0.3461394, NDCG: 0.1801037, F1: 0.0741051, HIT_RATE: 0.8284918, ARHR_ALL_HITS: 0.5181583, NOVELTY: 0.0053069, AVERAGE_POPULARITY: 0.6651199, DIVERSITY_MEAN_INTER_LIST: 0.7838044, DIVERSITY_HERFINDAHL: 0.9783747, COVERAGE_ITEM: 0.0455175, COVERAGE_ITEM_CORRECT: 0.0230356, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.8273993, DIVERSITY_GINI: 0.0035242, SHANNON_ENTROPY: 6.2162976, RATIO_DIVERSITY_HERFINDAHL: 0.9787422, RATIO_DIVERSITY_GINI: 0.0135947, RATIO_SHANNON_ENTROPY: 0.4995471, RATIO_AVERAGE_POPULARITY: 3.3522741, RATIO_NOVELTY: 0.0277612, 

Iteration No: 24 ended. Evaluation done at random point.
Time taken: 21.5244
Function

...

Iteration No: 70 ended. Search finished for the next optimal point.
Time taken: 22.9852
Function value obtained: -0.0868
Current minimum: -0.1203
Iteration No: 71 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.999014838667117, 'beta': 0.9087492704632686, 'gamma': 0.036917897469149315}
EvaluatorHoldout: Processed 13632 (100.0%) in 22.20 sec. Users per second: 614
SearchBayesianSkopt: Config 70 is suboptimal. Config: {'alpha': 0.999014838667117, 'beta': 0.9087492704632686, 'gamma': 0.036917897469149315} - results: PRECISION: 0.1801276, PRECISION_RECALL_MIN_DEN: 0.1813646, RECALL: 0.0459650, MAP: 0.0766404, MAP_MIN_DEN: 0.0769710, MRR: 0.3442142, NDCG: 0.1786392, F1: 0.0732405, HIT_RATE: 0.8257042, ARHR_ALL_HITS: 0.5141638, NOVELTY: 0.0053061, AVERAGE_POPULARITY: 0.6654242, DIVERSITY_MEAN_INTER_LIST: 0.7832074, DIVERSITY_HERFINDAHL: 0.9783150, COVERAGE_ITEM: 0.0448530, COVERAGE_ITEM_CORRECT: 0.0227587, COVERAGE_USER: 0.9986813, COVERAGE_USE

Iteration No: 77 ended. Search finished for the next optimal point.
Time taken: 22.0647
Function value obtained: -0.1203
Current minimum: -0.1203
Iteration No: 78 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.0, 'beta': 0.42155722434398996, 'gamma': 0.7750986706253207}
EvaluatorHoldout: Processed 13632 (100.0%) in 21.14 sec. Users per second: 645
SearchBayesianSkopt: Config 77 is suboptimal. Config: {'alpha': 0.0, 'beta': 0.42155722434398996, 'gamma': 0.7750986706253207} - results: PRECISION: 0.2472271, PRECISION_RECALL_MIN_DEN: 0.2515794, RECALL: 0.0708372, MAP: 0.1203164, MAP_MIN_DEN: 0.1220914, MRR: 0.4631195, NDCG: 0.2533466, F1: 0.1101217, HIT_RATE: 0.9225352, ARHR_ALL_HITS: 0.7443754, NOVELTY: 0.0056037, AVERAGE_POPULARITY: 0.5055989, DIVERSITY_MEAN_INTER_LIST: 0.9298958, DIVERSITY_HERFINDAHL: 0.9929828, COVERAGE_ITEM: 0.0882662, COVERAGE_ITEM_CORRECT: 0.0573121, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9213187, DIVERSIT

Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 22.4870
Function value obtained: -0.1202
Current minimum: -0.1203
Iteration No: 84 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.0, 'beta': 0.41130087613099237, 'gamma': 0.8595576942399411}
EvaluatorHoldout: Processed 13632 (100.0%) in 21.34 sec. Users per second: 639
SearchBayesianSkopt: Config 83 is suboptimal. Config: {'alpha': 0.0, 'beta': 0.41130087613099237, 'gamma': 0.8595576942399411} - results: PRECISION: 0.2473078, PRECISION_RECALL_MIN_DEN: 0.2516097, RECALL: 0.0706860, MAP: 0.1203362, MAP_MIN_DEN: 0.1221043, MRR: 0.4627478, NDCG: 0.2533326, F1: 0.1099468, HIT_RATE: 0.9227553, ARHR_ALL_HITS: 0.7441864, NOVELTY: 0.0056072, AVERAGE_POPULARITY: 0.5045945, DIVERSITY_MEAN_INTER_LIST: 0.9301649, DIVERSITY_HERFINDAHL: 0.9930097, COVERAGE_ITEM: 0.0904258, COVERAGE_ITEM_CORRECT: 0.0586965, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9215385, DIVERSIT

Iteration No: 90 ended. Search finished for the next optimal point.
Time taken: 22.5291
Function value obtained: -0.1203
Current minimum: -0.1203
Iteration No: 91 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.1199303813084415, 'beta': 0.015942073173547237, 'gamma': 0.006746827583013816}
EvaluatorHoldout: Processed 13632 (100.0%) in 21.28 sec. Users per second: 641
SearchBayesianSkopt: Config 90 is suboptimal. Config: {'alpha': 0.1199303813084415, 'beta': 0.015942073173547237, 'gamma': 0.006746827583013816} - results: PRECISION: 0.1776849, PRECISION_RECALL_MIN_DEN: 0.1787797, RECALL: 0.0447615, MAP: 0.0751271, MAP_MIN_DEN: 0.0754204, MRR: 0.3379140, NDCG: 0.1756494, F1: 0.0715088, HIT_RATE: 0.8171215, ARHR_ALL_HITS: 0.5043884, NOVELTY: 0.0053049, AVERAGE_POPULARITY: 0.6662325, DIVERSITY_MEAN_INTER_LIST: 0.7810868, DIVERSITY_HERFINDAHL: 0.9781030, COVERAGE_ITEM: 0.0449637, COVERAGE_ITEM_CORRECT: 0.0224265, COVERAGE_USER: 0.9986813, COVERA

Iteration No: 97 ended. Search finished for the next optimal point.
Time taken: 22.5265
Function value obtained: -0.1203
Current minimum: -0.1203
Iteration No: 98 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.0, 'beta': 0.4562475733311003, 'gamma': 0.8354679100549189}
EvaluatorHoldout: Processed 13632 (100.0%) in 21.39 sec. Users per second: 637
SearchBayesianSkopt: Config 97 is suboptimal. Config: {'alpha': 0.0, 'beta': 0.4562475733311003, 'gamma': 0.8354679100549189} - results: PRECISION: 0.2472198, PRECISION_RECALL_MIN_DEN: 0.2515721, RECALL: 0.0708316, MAP: 0.1203221, MAP_MIN_DEN: 0.1220976, MRR: 0.4631665, NDCG: 0.2533522, F1: 0.1101141, HIT_RATE: 0.9226819, ARHR_ALL_HITS: 0.7444214, NOVELTY: 0.0056035, AVERAGE_POPULARITY: 0.5056638, DIVERSITY_MEAN_INTER_LIST: 0.9298713, DIVERSITY_HERFINDAHL: 0.9929803, COVERAGE_ITEM: 0.0881001, COVERAGE_ITEM_CORRECT: 0.0573121, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9214652, DIVERSITY_

In [8]:
hybrid = MergeThreeModelsByScores(URM_train= ICM_combined, recommenders=recommenders)
hybrid.fit(alpha=0.9236610693606594, beta=0.31737234692557365, gamma=0.01980260938401746)

In [9]:
from Utils.utils import create_submission, write_submission

submission = create_submission(hybrid)
write_submission(submission,"SLIME_IALS_USERCF_Stacked_hybridBeta")